In [7]:
# Getting all FLOC from myANALYTIC where CompanyCode = 5000 -> df
import pandas as pd
import pyodbc

sql_query = """
    WITH dim_floc AS (
        SELECT
            FLOC_STRNO_FunctionalLocation,
            FLOC_PLTXT_FunctionalLocationDescr,
            FLOC_TPLNR_FunctionalLocationInternalKey,
            FLOC_EARTX_TechnicalObjectTypeDescr,
            FLOC_MSGRP_FunctionalLocationRoom,
            FLOC_STORT_LocationDescr,
            FLOC_BUKRS_CompanyCode
        FROM [myANALYTICS_SP].[silver.dimension.enterprise.asset].[vw_Dim_FunctionalLocation]
    ),

    fact_work_order AS (
        SELECT
            AUFNR_Order,
            GLTRP_BasicFinishDateID,
            KTEXT_OrderDescr,
            ILART_MaintenanceActivityTypeID,
            TPLNR_FunctionalLocationInternalID,
            ActualCost
        FROM [myANALYTICS_SP].[silver.fact.enterprise.asset].[vw_Fact_WorkOrder]
        WHERE ILART_MaintenanceActivityTypeID IN
            (
                'C01', 'C14', 'C20', 'C02', 'C13', 'C03', 'C19', 'C23', 'C25', 'C26', 'C10', -- Mechanised Track Maintenance
                'C29', 'C37', 'C08', 'C47', 'C54', 'C43', 'C51', 'C53', 'C57', 'C50', 'C10', 'C30', 'C48', 'C54', 'C52', 'C06', 'C07', 'C44', 'C01', 'C57', -- General Track Maintenance
                'B50', 'B53', 'C67', 'B04', 'B05', 'B06', 'B55', 'B57', 'C67', -- Structures Maintenance
                'T28', 'T29', 'T40', 'T41', 'T58', 'T44', 'T45', 'T46', 'T47', 'T48', 'T54', 'T42', 'T43', -- Control Systems - Signalling and Wayside Maintenance
                'T10', 'T11', 'T32', -- Control Systems - Telecommunications Maintenance
                'C54', 'T32', -- Control Systems - Operational Systems Maintenance
                'T26', 'T27', 'T32', 'T24', 'T25' -- Traction Power Maintenance
            )
    )

    SELECT
        dim_floc.FLOC_STRNO_FunctionalLocation AS 'floc',
        fact_work_order.AUFNR_Order AS 'work_order_id',
        --TRY_CAST(fact_work_order.GLTRP_BasicFinishDateID AS date) AS 'work_order_basic_finish_date',
        fact_work_order.ILART_MaintenanceActivityTypeID AS 'activity_type',
        fact_work_order.ActualCost as 'actual_cost'
    FROM dim_floc
        LEFT JOIN fact_work_order ON fact_work_order.TPLNR_FunctionalLocationInternalID = dim_floc.FLOC_TPLNR_FunctionalLocationInternalKey
        -- LEFT JOIN dim_work_order ON something
    where
        dim_floc.FLOC_BUKRS_CompanyCode = '5000' -- CompanyCode 5000 = Aurizon Network Pty Ltd (below rail)
        --AND fact_work_order.TPLNR_FunctionalLocationInternalID is not null -- this line will filter the result table to only show FLOC_IDs that HAD an order performed on it before (there's a lot of FLOC that weren't maintained before)
        --fact_work_order.RowNum = 1
    ORDER BY floc
"""

# Define your server name
server_name = 'myanalytics.aurizon.com.au'

# Establish a connection using Windows Authentication
conn = pyodbc.connect('DRIVER={SQL Server};SERVER=' + server_name + ';Trusted_Connection=yes;')

# Execute the SQL query and load the result into a pandas DataFrame
df1 = pd.read_sql_query(sql_query, conn)

# Display the DataFrame
df1.head(10)

C:\Users\r893859\AppData\Local\Temp\ipykernel_19340\3937003542.py:62: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df1 = pd.read_sql_query(sql_query, conn)


,floc,work_order_id,activity_type,actual_cost
0,AC-01CH,None,None,NaN
1,AC-01JN,None,None,NaN
2,AC-01JN-JNJN,None,None,NaN
3,AC-01NA,None,None,NaN
4,AC-01PA,None,None,NaN
5,AC-01RS,None,None,NaN
6,AC-02CH,None,None,NaN
7,AC-02JN,None,None,NaN
8,AC-02JN-JNJN,None,None,NaN
9,AC-02NA,None,None,NaN


In [8]:
df1.to_pickle("pkl/asset_maintenance_cost_2024_03_20.pkl")